# Processing the CoreLogic Data using PySpark

In this Jupyter notebook, we demonstrate how you can process the CoreLogic data using PySpark. 

In particular, we will show you how to:
* import the data, 
* explore rows and columns, 
* filter for a given location, 
* inspect the data for missing and erroneous information, 
* correct data errors,
* and write a subset of the data to a csv file for later use. 

This [PySpark cheat sheet](http://datacamp-community-prod.s3.amazonaws.com/acfa4325-1d43-4542-8ce4-bea2d287db10) provides a great overview of available PySpark functionality. 

The notebook was created using a 'Jupyter + Spark Basic session' in [Open on Demand](https://arc.umich.edu/open-ondemand/) (OOD) on [Great Lakes](https://arc.umich.edu/greatlakes/) (GL). This automatically initializes Spark in the background.

If you are not running the notebook using OOD on GL, you will most likely have to make sure PySpark is installed on your system and then initialize it by typing 

```from pyspark import SparkContext```

```sc = SparkContext(master = 'local[2]')```

If you encounter any errors or have questions about this Jupyter notebook, or if you would like us to add another PySpark example using the CoreLogic data, feel free to reach out to [Armand Burks](mailto:arburks@umich.edu) and [Jule Krüger](mailto:julianek@umich.edu).

## Importing the CoreLogic data with PySpark

The CoreLogic data are stored in a Turbo volume on the ```/nfs``` drive. To get access, you will need to sign a Memorandum of Understanding (MOU) with the University of Michigan Library. For more information about this, see [here](https://github.com/arc-ts/corelogic-on-greatlakes/tree/main/intro-to-corelogic-data). To execute this notebook, you will have to be granted access to the CoreLogic data on Turbo.

The raw data come in three separate files: deeds (28GB), foreclosures (6GB), and taxes (24GB). [CSCAR](https://cscar.research.umich.edu/) pre-processed the CoreLogic data and stored each raw file in 100 separate partitions. You can read more about this methodology in ```nfs/turbo/lib-data-corelogic/Docs/cscar_data.txt```.

We are going to work with the pre-processed data to improve import speeds. Let's store the paths to the pre-processed partitioned files in three separate variables:

In [1]:
turbo_path_fcl = "/nfs/turbo/lib-data-corelogic/Data/fcl/*.gz"
turbo_path_deed = "/nfs/turbo/lib-data-corelogic/Data/deed/*.gz"
turbo_path_tax = "/nfs/turbo/lib-data-corelogic/Data/tax/*.gz"

The following command imports a set of partitioned CoreLogic data files into a dataframe (df). You can switch between foreclosures, deeds, and taxes by choosing the relevant path accordingly. In each text file, the pipe ```|``` was used as a delimiter and is set  in the ```sep``` argument as such.

__A note on import speeds__: Because of their size, it takes a little while to read in each partitioned data file collection. Importing the raw data files takes much longer. Using the pre-processed data from CSCAR, which distributes the data across 100 partitions, greatly improves import speeds as Spark can spread computation across multiple cores.

Let's work with the foreclosure data for now:

In [2]:
df = spark.read.csv(turbo_path_fcl, header=True, sep="|")

(A small note on reading in the data: You could set the ```inferSchema=True``` argument, which would make Python infer the column type. The drawback of this is that FIPS and ZIP codes that start with a `0` have the leading `0` removed in the creation of an integer variable. We choose to keep ```inferSchema=False```, the default argument, to avoid this.)

## Exploring the CoreLogic data (rows and columns)

First, we want to get a basic idea of the CoreLogic data. What are the columns and what types of values do they contain? The .printSchema() method prints the data type (integer, string, double) for each column in the dataset. 

Because we set ```inferSchema=False```, all columns are read in as string data types. ```nullable = true``` means that a given column can accept missing (```null```) values.

In [3]:
df.printSchema()

root
 |-- APN: string (nullable = true)
 |-- FIPS: string (nullable = true)
 |-- State: string (nullable = true)
 |-- County: string (nullable = true)
 |-- BatchDateSeq Number: string (nullable = true)
 |-- DeedCategory: string (nullable = true)
 |-- DocumentType: string (nullable = true)
 |-- RecordingDate: string (nullable = true)
 |-- DocumentYear: string (nullable = true)
 |-- DocumentNumber: string (nullable = true)
 |-- DocumentBook: string (nullable = true)
 |-- DocumentPage: string (nullable = true)
 |-- TitleCompanyCode: string (nullable = true)
 |-- TitleCompanyName: string (nullable = true)
 |-- AttorneyName: string (nullable = true)
 |-- AttorneyPhoneNumber: string (nullable = true)
 |-- 1stDefendantBorrowerOwnerFirstName: string (nullable = true)
 |-- 1stDefendantBorrowerOwnerLastName: string (nullable = true)
 |-- 1stDefendantBorrowerOwnerCompanyName: string (nullable = true)
 |-- 2ndDefendantBorrowerOwnerFirstName: string (nullable = true)
 |-- 2ndDefendantBorrowerOwnerL

Let's calculate how many rows and columns there are in total. Note that we will store these values in their own variables because it takes quite a while to calculate the number of rows in our data. If we commit this number to current memory by storing it in a variable, we can easily reuse it later for various purposes without having to recalculate it first. This practice is also known as "lazy execution."

In [4]:
numRows = df.count()
numCols = len(df.columns)
print(numRows,numCols)

37487908 134


We can print a list of the column names like so.

In [5]:
df.columns

['APN',
 'FIPS',
 'State',
 'County',
 'BatchDateSeq Number',
 'DeedCategory',
 'DocumentType',
 'RecordingDate',
 'DocumentYear',
 'DocumentNumber',
 'DocumentBook',
 'DocumentPage',
 'TitleCompanyCode',
 'TitleCompanyName',
 'AttorneyName',
 'AttorneyPhoneNumber',
 '1stDefendantBorrowerOwnerFirstName',
 '1stDefendantBorrowerOwnerLastName',
 '1stDefendantBorrowerOwnerCompanyName',
 '2ndDefendantBorrowerOwnerFirstName',
 '2ndDefendantBorrowerOwnerLastName',
 '2ndDefendantBorrowerOwnerCompanyName',
 '3rdDefendantBorrowerOwnerFirstName',
 '3rdDefendantBorrowerOwnerLastName',
 '3rdDefendantBorrowerOwnerCompanyName',
 '4thDefendantBorrowerOwnerFirstName',
 '4thDefendantBorrowerOwnerLastName',
 '4thDefendantBorrowerOwnerCompanyName',
 'DefendantBorrowerOwnerEtAlIndicator',
 'Filler1',
 'DateofDefault',
 'AmountofDefault',
 'Filler2',
 'FilingDate',
 'CourtCaseNumber',
 'LisPendensType',
 'Plaintiff1/Seller',
 'Plaintiff2/Seller',
 'FinalJudgmentAmount',
 'Filler3',
 'AuctionDate',
 'Auction

## Inspecting variable distributions

We can use the ```describe``` method to calculate summary statistics for select columns (variables). Summary statistics can only be calculated for columns of type ```integer``` or ```double```. Currently, all of our variables are stored as type ```string```. We can convert a column to numeric values by casting it to type ```integer```.

In [6]:
from pyspark.sql.types import IntegerType
df = df.withColumn("MORTGAGE AMOUNT", df["MORTGAGE AMOUNT"].cast(IntegerType()))
df.select(["MORTGAGE AMOUNT"]).dtypes

[('MORTGAGE AMOUNT', 'int')]

Now that the 'mortgage amount' column is numeric, we can calculate basic summary statistics (mean, standard deviation, minimum and maximum value). As you can see, the average mortgage amount in the CoreLogic data is about $800K.

In [7]:
df.describe(['MORTGAGE AMOUNT']).show()

+-------+--------------------+
|summary|     MORTGAGE AMOUNT|
+-------+--------------------+
|  count|           111058818|
|   mean|   779943.3151940263|
| stddev|1.6808331794028625E7|
|    min|                   0|
|    max|          2075000000|
+-------+--------------------+



Let's do the same for the sale amount of a property:

In [8]:
df = df.withColumn("SALE AMOUNT", df["SALE AMOUNT"].cast(IntegerType()))
df.describe(['SALE AMOUNT']).show()

+-------+-----------------+
|summary|      SALE AMOUNT|
+-------+-----------------+
|  count|        213239549|
|   mean|402407.0318453403|
| stddev|6648148.318833588|
|    min|                0|
|    max|       2128327250|
+-------+-----------------+



Interestingly, the average sale amount is about half of the average mortgage amount. But these differences could also be driven by missing information in the data.

## Exploring the amount of missing data within and across columns

It is important to know for columns of interest how many missing values they contain. Information on data missingness guides decisions on what needs to be done with affected rows. Can rows with missing information simply be dropped? Or, do we need to try to fill in missing values conditional on information in other columns?

Let's investigate whether the mortgage column contains any missing values.

In [9]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan('MORTGAGE AMOUNT'),True))]).show()

+-----------------------------------------------------+
|count(CASE WHEN isnan(MORTGAGE AMOUNT) THEN true END)|
+-----------------------------------------------------+
|                                                    0|
+-----------------------------------------------------+



This column does not contain any missing values encoded as ```NaN```. Let's investigate whether the mortgage column containts any ```null``` values. This is just another way of how missing values could be stored.

In [10]:
df.select([count(when(col('MORTGAGE AMOUNT').isNull(),True))]).show()

+--------------------------------------------------------+
|count(CASE WHEN (MORTGAGE AMOUNT IS NULL) THEN true END)|
+--------------------------------------------------------+
|                                               256723662|
+--------------------------------------------------------+



That is quite a lot of missing information. We can calculate the percentage of ```null``` values in the mortgage column to get a better idea of the amount of missingness. To do so, we divide the number of rows with ```null``` values in the mortgage amount column by the total number of rows in the CoreLogic data. We can reuse the ```numRows``` variable here that we calculated earlier (see comment on 'lazy execution' above) to save us some processing time.

In [11]:
df.select([(count(when(col('MORTGAGE AMOUNT').isNull(),True))/numRows)]).show()

+----------------------------------------------------------------------+
|(count(CASE WHEN (MORTGAGE AMOUNT IS NULL) THEN true END) / 367782480)|
+----------------------------------------------------------------------+
|                                                    0.6980312439026459|
+----------------------------------------------------------------------+



As you can see, almost 70% of mortgage amount information is missing in the deeds dataset.

Let's calculate the proportion of records for which mortgage AND sale amount information is missing:

In [12]:
df.select([(count(when(col('MORTGAGE AMOUNT').isNull() & col('SALE AMOUNT').isNull(), True))/numRows)]).show()

+--------------------------------------------------------------------------------------------------+
|(count(CASE WHEN ((MORTGAGE AMOUNT IS NULL) AND (SALE AMOUNT IS NULL)) THEN true END) / 367782480)|
+--------------------------------------------------------------------------------------------------+
|                                                                               0.40302083720790616|
+--------------------------------------------------------------------------------------------------+



As we can see, about 40% of records have missing information on both the mortgage and sale amount columns.

## Inspecting categorical variables

We might be interested in the range of unique values of categorical variables. Let's print the total number of unique values of states designated by the 'SITUS STATE' column. (According to the Oxford Dictionary, "situs" is a legal term that designates "*the place to which, for purposes of legal jurisdiction or taxation, a property belongs.*")

In [13]:
len(df.select('SITUS STATE').distinct().rdd.map(lambda r: r[0]).collect())

53

The SITUS STATE column contains 53 unique values, which is unexpected. There should be 50 states plus Washington, DC. 

Let's create a frequency table of the column values to get a better idea of this variable. We will sort the table in descending order of state counts. We will set ```n=53``` in the show() method to print all rows in the table. The two additional values are: (1) ```null``` (the amount of missing values in the SITUS STATE column), and (2) `VI`, which stands for the Virgin Islands.

In [14]:
from pyspark.sql.functions import desc
df.groupBy('SITUS STATE').count().sort(desc("count")).show(n=53)

+-----------+--------+
|SITUS STATE|   count|
+-----------+--------+
|         CA|52167596|
|       null|35371569|
|         FL|35281257|
|         TX|21674192|
|         OH|13347704|
|         PA|12254911|
|         AZ|12006118|
|         IL|11947950|
|         GA|11148302|
|         NY|10456027|
|         NC| 9686809|
|         MI| 8638900|
|         TN| 8638649|
|         CO| 8112459|
|         WA| 8104844|
|         VA| 7827533|
|         NJ| 7320302|
|         MD| 6525203|
|         MO| 6496410|
|         SC| 6311502|
|         OK| 5217718|
|         MA| 4876134|
|         OR| 4767128|
|         NV| 4712444|
|         IN| 4394358|
|         AL| 4379161|
|         AR| 4378459|
|         MN| 4267855|
|         WI| 3864221|
|         UT| 3154019|
|         KY| 3018343|
|         MS| 2978654|
|         HI| 2572320|
|         CT| 2513164|
|         LA| 2497956|
|         WV| 2374639|
|         KS| 2348984|
|         IA| 1982068|
|         NM| 1546001|
|         NE| 1365483|
|         M

## Inspecting time variables

We might be interested in columns that present information about time/dates.

Let's identify the columns that seem to contain date information. As we can see below, dates are stored in the format ```YYYYMMDD``` or ```YYYY-MM-DD```.

In [17]:
df.select([s for s in df.columns if 'Date' in s]).show()

+-------------------+-------------+-------------+----------+-----------+----------------+-------------------------+------------------------+
|BatchDateSeq Number|RecordingDate|DateofDefault|FilingDate|AuctionDate|OriginalLoanDate|OriginalLoanRecordingDate|LastFullSaleTransferDate|
+-------------------+-------------+-------------+----------+-----------+----------------+-------------------------+------------------------+
| 201503180000000002|     20150317|         null|  20150304|   20150507|            null|                     null|              2000-06-02|
| 201605050000000001|     20160503|         null|  20160406|   20160526|            null|                     null|                    null|
| 193001010000021584|         null|         null|      null|       null|            null|                     null|                    null|
| 201506030000000054|     20150521|         null|  20150424|       null|        20070703|                 20070718|              2007-07-03|
| 19300101000

Let's work with the ```RecordingDate``` column to identify the time period covered in the CoreLogic data.

In [23]:
from pyspark.sql.functions import to_date
df.select('RecordingDate'.to_date('RecordingDate', "YYYYMMDD").alias("recording_date"))

AttributeError: 'str' object has no attribute 'to_date'

## Filtering the CoreLogic data to create a subset of observations

Suppose we only needed a subset of the CoreLogic data for research purposes, i.e., we were interested in property information in a select location. 

Here, our goal is to filter the CoreLogic data to create a subset that only contains rows relating to Detroit, Michigan. 

To start, let's print a select few rows and columns of the dataframe:

In [15]:
df.select('FIPS', 'SITUS CITY', 'SITUS STATE', 'SITUS ZIP CODE').show()

+-----+----------+-----------+--------------+
| FIPS|SITUS CITY|SITUS STATE|SITUS ZIP CODE|
+-----+----------+-----------+--------------+
|12099|      null|       null|          null|
|17097|      null|       null|          null|
|97200|      null|       null|          null|
|97200|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|97199|      null|       null|          null|
|17097|      null|       null|    

Notice how there are many rows containing columns with ```null``` values. We will remove rows that have ```null``` values across all columns of interest because it would be hard to deduce the location without any of this information (FIPS code, property city, state and zip code) present.

In [16]:
df = df.dropna(subset=["FIPS","SITUS CITY", "SITUS STATE", "SITUS ZIP CODE"])

Now let's see how many rows we have left out of the original 367,782,480:

In [17]:
df.count()

311359183

Let's assume we wanted to filter the CoreLogic data for Wayne county, MI, where the city of Detroit is located. The FIPS code for Wayne county is '26163' ([source](https://mi.postcodebase.com/county/26163)).

In [18]:
WayneCsubset = df.filter(df.FIPS==26163)

Let's print a few rows and columns of the subset. As you can see, we are only dealing with Wayne county now.

In [19]:
WayneCsubset.select('FIPS', 'SITUS CITY', 'SITUS STATE', 'SITUS ZIP CODE').show()

+-----+----------+-----------+--------------+
| FIPS|SITUS CITY|SITUS STATE|SITUS ZIP CODE|
+-----+----------+-----------+--------------+
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482013148|
|26163|   DETROIT|         MI|     482012463|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|     482022828|
|26163|   DETROIT|         MI|         48202|
|26163|   DETROIT|         MI|     482021302|
|26163|   DETROIT|         MI|     482021368|
|26163|   DETROIT|         MI|     482021368|
|26163|   DETROIT|         MI|     482021368|
|26163|   DETROIT|         MI|    

How many rows are in the Wayne county subset? We can use the count() method:

In [20]:
WayneCsubset.count()

2132959

There are many different cities within Wayne county. Let's get a list of all of them:

In [21]:
WayneCsubset.select('SITUS CITY').distinct().collect()

[Row(SITUS CITY='LINCOLN PARK'),
 Row(SITUS CITY='MELVINDALE'),
 Row(SITUS CITY='NORTHVILLE'),
 Row(SITUS CITY='TRENTON'),
 Row(SITUS CITY='YPSILANTI'),
 Row(SITUS CITY='BELLEVILLE'),
 Row(SITUS CITY='OAK PARK'),
 Row(SITUS CITY='RIVERVIEW'),
 Row(SITUS CITY='GROSSE POINTE WOODS'),
 Row(SITUS CITY='FLAT ROCK'),
 Row(SITUS CITY='GROSSE ILE'),
 Row(SITUS CITY='GROSSE POINTE PARK'),
 Row(SITUS CITY='WARREN'),
 Row(SITUS CITY='GIBRALTAR'),
 Row(SITUS CITY='GROSSE POINTE'),
 Row(SITUS CITY='MONROE'),
 Row(SITUS CITY='WYANDOTTE'),
 Row(SITUS CITY='HIGHLAND'),
 Row(SITUS CITY='WOODHAVEN'),
 Row(SITUS CITY='INKSTER'),
 Row(SITUS CITY='SOUTHGATE'),
 Row(SITUS CITY='SUMPTER TWP'),
 Row(SITUS CITY='GARDEN CITY'),
 Row(SITUS CITY='HIGHLAND PARK'),
 Row(SITUS CITY='DETROIT'),
 Row(SITUS CITY='FERNDALE'),
 Row(SITUS CITY='CHELSEA'),
 Row(SITUS CITY='BROWNSTOWN TWP'),
 Row(SITUS CITY='NEW HUDSON'),
 Row(SITUS CITY='LANSING'),
 Row(SITUS CITY='DEARBORN HEIGHTS'),
 Row(SITUS CITY='CANTON'),
 Row(SITUS 

Let's create a subset for Detroit only. We will filter using the 'SITUS CITY' variable. This variables designates the city associated with the property address. 

Note that this is a straightforward filtering method if using the Wayne county subset we created previously. 

In [22]:
detroit = WayneCsubset.filter(WayneCsubset["SITUS CITY"]=="DETROIT")

If you wanted to create a Detroit subset from the entire dataset, we suggest filtering on 'SITUS CITY' and 'SITUS STATE'. When using the entire dataframe, filtering on 'SITUS CITY' alone could run into city name ambiguity across states. 

In [23]:
detroit2 = df.filter((df["SITUS CITY"]=="DETROIT") & (df["SITUS STATE"]=="MI"))

How many rows are in the Detroit subset? Depending on the size of the dataframe, count() can take a bit of time to calculate. Therefore, we will save the value to a variable in case we need to use it later on. This can save us some processing time (cf. comment on "lazy evaluation" above).

In [24]:
detroitCount = detroit.count()
detroitCount

1129407

Do the two filtering methods yield the same number of rows?

In [25]:
detroit2Count = detroit2.count()
detroit2Count

1129438

In [26]:
detroit2Count - detroitCount

31

The second subset contains slightly more records. This means that we get two different results depending on whether we filter by (SITUS CITY == "DETROIT" AND SITUS STATE == "MI") versus (FIPS == "26163" AND SITUS CITY == "DETROIT"). 

Let's investigate the Detroit subset with more records (```detroit2```). We can tabulate the FIPS codes that have city=="DETROIT" and state=="MI": 

In [27]:
detroit2.groupBy('FIPS', 'SITUS CITY', 'SITUS STATE').count().show()

+-----+----------+-----------+-------+
| FIPS|SITUS CITY|SITUS STATE|  count|
+-----+----------+-----------+-------+
|26147|   DETROIT|         MI|      6|
|26061|   DETROIT|         MI|      2|
|26035|   DETROIT|         MI|      2|
|26161|   DETROIT|         MI|      1|
|26163|   DETROIT|         MI|1129407|
|48039|   DETROIT|         MI|      2|
|26069|   DETROIT|         MI|      2|
|26071|   DETROIT|         MI|      5|
|26099|   DETROIT|         MI|      2|
|26115|   DETROIT|         MI|      9|
+-----+----------+-----------+-------+



As you can see, there are multiple FIPS codes associated with Detroit, MI, in this data subset when there should really only be one FIPS code for Detroit in Wayne County, Michigan.  For example, FIPS code '26115' (9 occurences) is for Monroe County which does not contain the city of Detroit. 

For each of the additional FIPS codes identified above, we would have to do some further research to determine whether these FIPS codes constitute data entry errors that need to be corrected or whether they identify properties that are not actually located in Detroit. To do this, we would have to review each FIPS code and also inspect the addresses associated with each property to determine where exactly a property is legally located. This work is beyond the purposes of the current analysis.

## Saving a subset of the CoreLogic data to a spreadsheet

Let's assume the detroit2 dataframe looks like what we are interested in and we now want to write this Detroit subset to a new csv file in an ```output/``` folder in our ```home/[uniqname]/``` directory. (Note: You probably need to change this path, depending on where you store this notebook.) 

By default, Spark writes big data into multiple files to optimize computation times during import and export. This is recommended practice, especially if the data is big, i.e., if it has many rows and/or columns.

The below command writes 100 partitions of the Detroit data (i.e., 100 csv files) to the designated folder. 

_Note_: If you saved data to this path before, you will need to add the argument ```overwrite=True``` to the function call like so ```detroit2.write.csv("../../output/corelogic_data_deeds_Detroit_partitioned", overwrite=True)```

In [28]:
detroit2.write.csv("../../output/corelogic_data_deeds_Detroit_partitioned")

There is also a way to force the data into one single file. Sometimes, a single file is needed to continue processing the data with other research software. Saving the data into one file is a little risky and might not always work, so caution is advised. 

The following command combines the data into one core (Caution: it is very slow). If the data does not fit, it will throw an error. _Note_: If you saved data to this path before, you will need to add the argument ```overwrite=True``` to the function call. The overwrite argument would overwrite the previously created ```output/``` directory.

Note: You would only need to run this, if you indeed need only one single data file. Many research programs allow you to read in and combine multiple data files into one object. See for example the [glob method in Python](https://docs.python.org/3/library/glob.html).

In [29]:
detroit2.coalesce(1).write.csv("../../output/corelogic_data_deeds_Detroit_singlefile")

## Drawing a data sample for research and development purposes
Let's say we want to draw and save just a sample of the Detroit data (n~1000) that is small enough to use for writing and testing code for analyzing the Detroit subset. We can use PySpark's DataFrame.sample() method. However, we have to specify a fraction of the dataframe size rather than explicitly the size of the sample, n. Also, we are not guaranteed to get EXACTLY the fraction that we specify, due to the way the sample() function is implemented. Let's see how that works.

Since we want about 1000 rows in our sample, we can divide 1000 by detroit2Count to see what fraction that is.

In [30]:
sampleFraction = 1000 / detroit2Count
sampleFraction

0.0008853960996531018

Now, let's sample the Detroit data.

In [31]:
detroitSample = detroit2.sample(fraction=sampleFraction)

How many records do we have in our sample?

In [32]:
detroitSample.count()

1002

Now that we have created a small sample of our Detroit records for research and development purposes, we can write the results to a spreadsheet (note that using coalesce(1) can still be quite expensive. Use only if absolutely necessary).

In [33]:
detroitSample.write.csv("../../output/corelogic_data_deeds_Detroit_sampled_1031_singlefile")